In [ ]:
from local_gemma import LocalGemma2ForCausalLM
from transformers import AutoTokenizer

model = LocalGemma2ForCausalLM.from_pretrained("google/gemma-2-27b-it", preset="auto")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-27b-it")

def gemma_chat(model=model, tokenizer=tokenizer, messages=[]):

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

    generated_ids = model.generate(**model_inputs.to(model.device), max_new_tokens=1024, do_sample=True)

    # Get only the new tokens generated by the model (excluding the input)
    input_length = model_inputs["input_ids"].shape[1]
    generated_response_ids = generated_ids[:, input_length:]

    # Decode only the generated response
    decoded_response = tokenizer.batch_decode(generated_response_ids, skip_special_tokens=True)[0]

    return decoded_response

In [ ]:
import abc

class Agent(abc.ABC):
    def __init__(self):
        self.user_initial_message = {'role': 'user', 'content': "Introduce yourself."}
        self.sys_message = {'role': 'assistant', 'content': self.system_prompt()}
        self.user_message_default = {'role': 'user', 'content': 'Give me a demonstration.'}

    @abc.abstractmethod
    def system_prompt(self):
        """Return the system prompt content"""
        pass

    @abc.abstractmethod
    def user_prompt(self):
        pass

    def chat(self, user_input=None, model=model):

        if user_input:
            user_content = self.user_prompt(user_input)
        else:
            user_content = self.user_message_default

        #print(user_content)

        user_message = {'role': 'user', 'content': user_content}

        messages = [self.user_initial_message, self.sys_message, user_message]

        print(f'Passing the following messages to model: {messages}')

        response = gemma_chat(model=model, messages=messages)

        return response.strip()


In [ ]:
class Preprocessor(Agent):
    def system_prompt(self):
        return "I am an expert editor of Tibetan Buddhist texts. I preprocess Tibetan material for translation."
    
    def user_prompt(self, user_input):
        return f"""Preprocess the following: {user_input}
                """

In [ ]:
class Translator(Agent):
    def system_prompt(self):
        return "I am an expert translator of Tibetan Buddhist texts. I translate clearly and concisely, preserving the original intent of the text."
    
    def user_prompt(self, user_input):
        return f"Translate the following from Tibetan to English:{user_input}"


In [ ]:
class Evaluator(Agent):
    def system_prompt(self):
        return "I am an expert in translation evaluation. I score translations based on fluency, literary quality, and meaning preservation."
    
    def user_prompt(self, user_input):
        return f"""Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar".

                Source: {user_input['source']}
                Target: {user_input['target']}
                """

In [ ]:
class PostEditor(Agent):
    def system_prompt(self):
        return """"I am an expert translation editor. I provide improved translations based on an original translation and a score from an evaluator. 
        I preserve the good qualities of the first draft but improve clarity and fluency."""
    
    def user_prompt(self, user_input):
        return f"""Improv the following translation:
        Source: {user_input['source']},
        Translation: {user_input['translation']}
        Score: {user_input['score']}
        """

In [ ]:
prepro = Preprocessor()
translator = Translator()
eval = Evaluator()
post = PostEditor()

In [ ]:
tib = """སེམས་ཅན་ཐམས་ཅད་ཀྱི་སྡུག་བསྔལ་སེལ་བ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་སྡུག་བསྔལ་ལས་གྲོལ་བར་མཛད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ཀྱི་བཅིངས་པ་གཅོད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ལ་མི་འཇིགས་པ་སྦྱིན་པ་ལ་ཕྱག་འཚལ་ལོ། །"""

eng = """Homage to the one who dispels the suffering of all beings.
Homage to the one who liberates all beings from suffering.
Homage to the one who cuts the bonds of all beings.
Homage to the one who grants fearlessness to all beings."""

In [ ]:
corrupted_tib = """སེམས་ཅན་ཐམས་ཅད་ཀྱི་སྡུག་བསལ་བ་ལ་ཕྱག་འཚལ་ལོ། །
སེམསན་ཐམས་ཅད་སྡུག་བསྔལ་ལས་གྲོལ་བར་མཛད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ཀྱི་བཅ་པ་གཅོད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ལ་མི་འཇིགས་པ་སྦྱིན་པ་ལ་ཕག་འཚལ་ལོ། །"""

prepro.chat(user_input=corrupted_tib)

In [ ]:
response = translator.chat(user_input=tib)
response

In [ ]:
eval.chat({'source': tib, 'target': response})

In [ ]:
post.chat({'source': tib, 'translation': response , 'score':50})